# Model performance

Creating a baseline to compare against current model performance.

In [1]:
import itertools
import json
import yaml

import numpy as np
import pandas as pd
from scipy import optimize

In [2]:
# Move project root folder
%cd ..

/Users/mitchell/projects/personal/f1-rating-system


/Users/mitchell/Library/Caches/pypoetry/virtualenvs/f1-rating-system-bj5GO79q-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
with open("params.yaml") as conf_file:
    CONFIG = yaml.safe_load(conf_file)

# Baseline model: Elo model

In [14]:
class EloRatingSystem():
    '''Baseline rating system for F1 drivers'''

    def __init__(self, k: float, c: float):
        self.k = k
        self.c = c

    def get_win_prob(self, rating_a: float, rating_b: float) -> float:
        '''Returns the win probability of driver-constructor A over driver-constructor B'''
        return 1 / (1 + np.exp(-(rating_a - rating_b) / self.c))
    
    def get_rating_change(self, rating_change: float) -> float:
        '''Returns updated driver rating'''
        return self.k* rating_change

In [30]:
# column indexes
DRI_IX = 5 # driver id
POS_IX = 6 # driver position

MOD_DF = pd.read_csv(CONFIG["data"]["features_path"])
MOD_DF[["constructorScore", "driverScore", "expected", "actual"]] = None
IX_CHUNKS = MOD_DF.reset_index().groupby(["year", "round"])["index"].agg(["min", "max"]).values
MOD_MAT = MOD_DF.values

DRI_RTG = {dri: CONFIG["model"]["start_score"] for dri in set(MOD_DF["driverId"])}

def model_data(params: dict) -> float:
    '''Returns mean negative log likelihood of the rating system. If
    export = True, also exports results for data reporting.'''

    dri_scores = DRI_RTG.copy()
    log_likelihood = 0
    n_pred = 0
    model = EloRatingSystem(params[0], params[1])

    for start_ix, end_ix in IX_CHUNKS:
        yr_mat = MOD_MAT[start_ix:end_ix+1]

        rnd_dri_scores = {dri: 0 for dri in yr_mat[:, DRI_IX]}

        for ix_1, ix_2 in itertools.combinations(range(yr_mat.shape[0]), 2):
            dri_a, pos_a = yr_mat[ix_1, [DRI_IX, POS_IX]]
            dri_b, pos_b = yr_mat[ix_2, [DRI_IX, POS_IX]]
    
            # continue if drivers in same car or a driver does not finish for misc reason
            if pos_a == pos_b:
                continue

            # get current rating
            elo_a = dri_scores[dri_a]
            elo_b = dri_scores[dri_b]
            
            # create expected scores
            e_a = model.get_win_prob(elo_a, elo_b)
            e_b = 1 - e_a

            # create true scores and track log likelihood 
            if pos_a < pos_b:
                o_a = 1
                log_likelihood += np.log(max(e_a, 1E-10))

            else:
                o_a = 0
                log_likelihood += np.log(max(e_b, 1E-10))

            n_pred += 1
                
            # calculate score change and update round scores
            diff_a = o_a - e_a
            diff_b = -diff_a

            # log driver score changes per round
            rnd_dri_scores[dri_a] += diff_a
            rnd_dri_scores[dri_b] += diff_b
        
        # update driver values for finishing drivers and driver-caused retirements
        for dri in rnd_dri_scores.keys():
            dri_scores[dri] += model.get_rating_change(rnd_dri_scores[dri])

    return - log_likelihood / n_pred

params = [
    32, # K-factor - sensitivity of rating change
    400 # C-factor - sensitivity of expected outcome
]
result = optimize.minimize(model_data, params, method="L-BFGS-B", options={"disp": True})

# 313266 predictions 

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.64508D-01    |proj g|=  3.12200D-03


 This problem is unconstrained.



At iterate    1    f=  6.49685D-01    |proj g|=  2.78665D-03

At iterate    2    f=  6.36940D-01    |proj g|=  2.42540D-03

At iterate    3    f=  6.17563D-01    |proj g|=  1.26630D-03

At iterate    4    f=  6.14379D-01    |proj g|=  1.78302D-04

At iterate    5    f=  6.14354D-01    |proj g|=  1.10378D-04

At iterate    6    f=  6.14341D-01    |proj g|=  8.17124D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      6     14      1     0     0   8.171D-06   6.143D-01
  F =  0.61434076350413436     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [31]:
print(f"Baseline model best parameters: {result.x}")
print(f"Baseline model neg log likelihood: {model_data(result.x)}")

Baseline model best parameters: [  7.57423467 398.36297615]
Baseline model neg log likelihood: 0.6143407635041344


In [32]:
with open("models/metrics.json") as infile:
    perf_dict = json.load(infile)

print(f"Current system neg log likelihood: {round(perf_dict['log_likelihood'], 4)}")

Current system neg log likelihood: 0.6095
